In [31]:
import pandas as pd

In [32]:
cases = pd.read_csv('cases.csv')
labs = pd.read_csv('labs.csv')

In [33]:
cases.columns

Index(['caseid', 'subjectid', 'casestart', 'caseend', 'anestart', 'aneend',
       'opstart', 'opend', 'adm', 'dis', 'icu_days', 'death_inhosp', 'age',
       'sex', 'height', 'weight', 'bmi', 'asa', 'emop', 'department', 'optype',
       'dx', 'opname', 'approach', 'position', 'ane_type', 'preop_htn',
       'preop_dm', 'preop_ecg', 'preop_pft', 'preop_hb', 'preop_plt',
       'preop_pt', 'preop_aptt', 'preop_na', 'preop_k', 'preop_gluc',
       'preop_alb', 'preop_ast', 'preop_alt', 'preop_bun', 'preop_cr',
       'preop_ph', 'preop_hco3', 'preop_be', 'preop_pao2', 'preop_paco2',
       'preop_sao2', 'cormack', 'airway', 'tubesize', 'dltubesize', 'lmasize',
       'iv1', 'iv2', 'aline1', 'aline2', 'cline1', 'cline2', 'intraop_ebl',
       'intraop_uo', 'intraop_rbc', 'intraop_ffp', 'intraop_crystalloid',
       'intraop_colloid', 'intraop_ppf', 'intraop_mdz', 'intraop_ftn',
       'intraop_rocu', 'intraop_vecu', 'intraop_eph', 'intraop_phe',
       'intraop_epi', 'intraop_ca'],
     

In [112]:
cases['region'] = cases['optype']

mean_by_case = labs.groupby(['caseid', 'name'])['result'].mean().reset_index()

reference_ranges = {
    'wbc': [4, 10],    'hb': [13, 17],   'hct': [39, 52],  'plt': [130, 400],
    'esr': [0, 9],     'gluc': [70, 110], 'tprot': [6.0, 8.0], 'alb': [3.3, 5.2],
    'tbil': [0.2, 1.2], 'ast': [1, 40],   'alt': [1, 40],   'bun': [10, 26],
    'cr': [0.70, 1.40], 'gfr': [90, 120], 'ccr': [75, 125],  'na': [135, 145],
    'k': [3.5, 5.5],    'ica': [1.05, 1.35], 'cl': [98, 110], 'ammo': [27.2, 102],
    'crp': [0, 0.5],    'lac': [0.5, 2.2], 'ptinr': [0.8, 1.2], 'pt%': [80, 120],
    'ptsec': [10.6, 12.9], 'aptt': [26.7, 36.6], 'fib': [192, 411],
    'ph': [7.35, 7.45], 'pco2': [35, 48], 'po2': [83, 108],
    'hco3': [18, 23.0], 'be': [-2.0, 3.0], 'sao2': [95, 98]
}

def normalize(row):
    rng = reference_ranges.get(row['name'])
    if rng and pd.notna(row['result']):
        mn, mx = rng
        val = (row['result'] - mn) / (mx - mn)
        return max(0, min(1, val))
    return None

mean_by_case['normalized'] = mean_by_case.apply(normalize, axis=1)

pivot = (
    mean_by_case
    .pivot(index='caseid', columns='name', values='normalized')
    .reset_index()
)

meta = (
    cases
    .groupby('caseid')
    .first()
    .reset_index()[['caseid','sex', 'age', 'height','weight', 'region']]
)

data = meta.merge(pivot, on='caseid')
data = data.dropna(how='all', axis=1)
data

,caseid,sex,age,height,weight,region,alb,alt,ammo,aptt,...,ph,plt,po2,pt%,ptinr,ptsec,sao2,tbil,tprot,wbc
0,1,M,77.0,160.2,67.50,Colorectal,0.000000,0.391026,NaN,0.131313,...,0.833333,0.169444,1.0,0.225000,0.700000,0.739130,1.0,0.650000,0.0625,1.000000
1,2,M,54.0,167.3,54.80,Stomach,0.008772,0.491453,NaN,0.191919,...,NaN,0.256085,NaN,0.600000,0.425000,0.217391,NaN,0.433333,0.0000,0.345000
2,4,M,74.0,160.6,53.00,Stomach,0.000000,0.487179,NaN,0.398990,...,0.183333,0.407870,1.0,0.000000,0.987500,1.000000,1.0,1.000000,0.0000,0.836667
3,5,M,66.0,171.0,59.70,Vascular,0.000000,0.898860,NaN,1.000000,...,0.310000,0.000000,1.0,0.122727,0.804545,0.944664,1.0,0.638889,0.0000,1.000000
4,6,F,78.0,150.0,54.60,Biliary/Pancreas,0.368421,0.581197,NaN,0.242424,...,NaN,0.082716,NaN,0.600000,0.425000,0.217391,NaN,0.700000,0.3500,0.033333
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5791,6384,M,64.0,161.5,63.00,Stomach,0.092105,0.750000,NaN,NaN,...,NaN,0.526852,NaN,NaN,NaN,NaN,NaN,0.475000,0.1375,0.451250
5792,6385,M,69.0,159.3,62.30,Colorectal,0.252632,1.000000,NaN,0.000000,...,0.833333,0.768889,1.0,0.575000,0.450000,0.260870,1.0,0.520000,0.2350,0.856667
5793,6386,F,61.0,151.7,43.25,Stomach,0.078947,0.820513,NaN,NaN,...,0.700000,0.224691,1.0,NaN,NaN,NaN,1.0,0.350000,0.1250,0.703333
5794,6387,F,24.0,155.7,55.50,Biliary/Pancreas,0.223684,0.448718,NaN,0.489899,...,1.000000,0.289815,1.0,0.062500,0.862500,1.000000,1.0,0.225000,0.0875,0.530833


In [113]:
data.to_json('kate_chart.json', orient='records')

In [111]:
cases['optype'].value_counts()

optype
Colorectal          1350
Biliary/Pancreas     812
Others               799
Stomach              676
Major resection      584
Minor resection      553
Breast               434
Transplantation      403
Vascular             262
Hepatic              258
Thyroid              257
Name: count, dtype: int64